**Data mining Project - University of Pisa, acedemic year 2023/24**

**Authors**: Giacomo Aru, Giulia Ghisolfi, Luca Marini, Irene Testa
# Selection of samples to use with explainers

In [1]:
import pandas as pd
import numpy as np
from explanation_utils import *
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)
DATA_DIR = '../data/classification_results/'

We load the test set and the predictions of the classifiers:

In [2]:
incidents_test_df = pd.read_csv('../data/clf_indicators_test.csv', index_col=0)
true_labels_test_df = pd.read_csv('../data/clf_y_test.csv', index_col=0)
true_labels_test = true_labels_test_df.values.ravel()

clf_names = [clf.value for clf in Classifiers]

preds = get_classifiers_predictions(DATA_DIR)

incidents_train_df = pd.read_csv('../data/clf_indicators_train.csv', index_col=0)
true_labels_train_df = pd.read_csv('../data/clf_y_train.csv', index_col=0)

In [3]:
(
    incidents_train_df[(incidents_train_df['suicide']==1) & (true_labels_train_df['death']==0)].shape[0] + 
    incidents_test_df[(incidents_test_df['suicide']==1) & (true_labels_test_df['death']==0)].shape[0]
    ) / \
(
    incidents_train_df[(incidents_train_df['suicide']==1) & (true_labels_train_df['death']==1)].shape[0] + 
    incidents_test_df[(incidents_test_df['suicide']==1) & (true_labels_test_df['death']==1)].shape[0]
)*100

6.262510723477266

In [4]:
100-((incidents_train_df[(incidents_train_df['suicide']==1) & (true_labels_train_df['death']==0)].shape[0])/(incidents_train_df[(incidents_train_df['suicide']==1)].shape[0])*100)

93.45831671320303

In [5]:
selected_records_to_explain = {}
selected_records_to_explain['indexes'] = []
selected_records_to_explain['positions'] = []
selected_records_to_explain['instance names'] = []
selected_records_to_explain['true labels'] = []

## Attempted suicides

In [6]:
attempted_suicides = incidents_test_df[
    (incidents_test_df['suicide']==1) &
    (true_labels_test_df['death']==0) &
    (incidents_test_df['n_participants']==1)
]
attempted_suicides

,date_original,state,address,county,city,address_type,state_house_district,state_senate_district,px_code,participant_age1,participant1_child,participant1_teen,participant1_adult,participant1_male,participant1_female,min_age,max_age,n_child,n_teen,n_adult,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,notes,incident_characteristics1,incident_characteristics2,candidate_votes,total_votes,candidate_perc,population_state_2010,location_imp,latitude,longitude,x,y,state_code,congressional_district,age_range,avg_age,n_child_prop,n_teen_prop,n_males_prop,n_participants,day,day_x,day_y,day_of_week,day_of_week_x,day_of_week_y,month,month_x,month_y,year,days_from_first_incident,poverty_perc,democrat,gun_law_rank,aggression,accidental,defensive,suicide,road,house,school,business,illegal_holding,drug_alcohol,officers,organized,social_reasons,abduction
178231,2016-04-28,TEXAS,5000 block of Texas 21,Cameron County,Brownsville,place,37.0,27.0,TX,37.0,False,False,True,True,False,37.0,37.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,Perp escapes from prison and steals a shotgun from his former boss and later shoots himself.,Shot - Wounded/Injured,Suicide - Attempt,104638.0,166961.0,62.672121,25145561,1.000000e-07,25.9119,-97.5043,6.498082e+05,2.866782e+06,43,34.0,0.0,37.0,0.0,0.0,1.0,1.0,28,-0.571268,0.820763,3,0.433884,-0.900969,4,8.660254e-01,-5.000000e-01,2016.0,1213,13.7,1.0,13.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
99415,2014-05-16,WASHINGTON,NaN,Spokane County,Spokane,amenity,3.0,3.0,WA,37.0,False,False,True,True,False,37.0,37.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,"Welfare check called in on suicidal man with gun wanting ""suicide by cop"". Police persuaded man to give up gun without incident.",Suicide - Attempt,Officer Involved Incident,135470.0,223242.0,60.683026,6724540,1.000000e-05,47.6588,-117.4260,-8.809601e+05,5.445264e+06,47,5.0,0.0,37.0,0.0,0.0,1.0,1.0,16,-0.101168,-0.994869,4,-0.433884,-0.900969,5,5.000000e-01,-8.660254e-01,2014.0,500,11.5,0.0,6.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
163739,2015-06-29,WASHINGTON,NE 13th Ave and NE 72nd St,Clark County,Vancouver,building,49.0,49.0,WA,26.0,False,False,True,False,True,26.0,26.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,"Woman was in a stolen vehicle w/her bf. Self-inflicted gunshot wound.;\r\r\r\r\n45.674055, -122.659362",Shot - Wounded/Injured,Suicide - Attempt,124796.0,202814.0,61.532241,6724540,1.000000e-05,45.6741,-122.6590,-1.340121e+06,5.337516e+06,47,3.0,0.0,26.0,0.0,0.0,0.0,1.0,29,-0.394356,0.918958,0,0.000000,1.000000,6,1.224647e-16,-1.000000e+00,2015.0,909,10.7,0.0,6.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
238644,2015-08-05,PENNSYLVANIA,1900 block of N Churchtown Rd,Lancaster County,Caernarvon Township,place,NaN,NaN,PA,35.0,False,False,True,True,False,35.0,35.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,"armed carjack at 1500 block of Reading Road; chase end, attempt suicide at N Churchtown Rd;\r\r\r\r\n40.140012, -75.981988",Shot - Wounded/Injured,Suicide - Attempt,101722.0,176235.0,57.719522,12702379,1.000000e-07,40.1402,-75.9830,2.468584e+06,4.705954e+06,38,16.0,0.0,35.0,0.0,0.0,1.0,1.0,5,0.848644,0.528964,2,0.974928,-0.222521,8,-8.660254e-01,-5.000000e-01,2015.0,946,11.5,0.0,8.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
236642,2016-08-01,INDIANA,5500 block of Manker Street,Marion County,Indianapolis,building,97.0,36.0,IN,30.0,False,False,True,True,False,30.0,30.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,"Man w/ known mental issues believed to have attempted suicide by cop, wounded; stable;",Shot - Wounded/Injured,Suicide - Attempt,158739.0,264670.0,59.976197,6483802,1.000000e-05,39.7264,-86.1420,1.603579e+06,4.477271e+06,14,7.0,0.0,30.0,0.0,0.0,1.0,1.0,1,0.201299,0.979530,0,0.000000,1.000000,8,-8.660254e-01,-5.000000e-01,2016.0,1308,11.8,1.0,12.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
140321,2016-03-13,NEW JERSEY,NaN,Essex County,Newark,amenity,29.0,29.0,NJ,39.0,False,False,True,True,False,39.0,39.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,Man attempts to commit suicide during standoff with police by shooting self in stomach.,Shot - Wounded

In [7]:
attempted_suicide_index = attempted_suicides.index[0]
attempted_suicide_pos = incidents_test_df.index.get_loc(attempted_suicide_index)

selected_records_to_explain['indexes'].append(attempted_suicide_index)
selected_records_to_explain['positions'].append(attempted_suicide_pos)
selected_records_to_explain['instance names'].append('Attempted Suicide')
selected_records_to_explain['true labels'].append(true_labels_test[attempted_suicide_pos])

## Mass shootings

In [8]:
max_killed = incidents_test_df['n_killed'].max()
mass_shooting = incidents_test_df[incidents_test_df['n_killed'] == max_killed]
mass_shooting

,date_original,state,address,county,city,address_type,state_house_district,state_senate_district,px_code,participant_age1,participant1_child,participant1_teen,participant1_adult,participant1_male,participant1_female,min_age,max_age,n_child,n_teen,n_adult,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,notes,incident_characteristics1,incident_characteristics2,candidate_votes,total_votes,candidate_perc,population_state_2010,location_imp,latitude,longitude,x,y,state_code,congressional_district,age_range,avg_age,n_child_prop,n_teen_prop,n_males_prop,n_participants,day,day_x,day_y,day_of_week,day_of_week_x,day_of_week_y,month,month_x,month_y,year,days_from_first_incident,poverty_perc,democrat,gun_law_rank,aggression,accidental,defensive,suicide,road,house,school,business,illegal_holding,drug_alcohol,officers,organized,social_reasons,abduction
16951,2015-06-17,SOUTH CAROLINA,110 Calhoun Street,Charleston County,Charleston,amenity,111.0,42.0,SC,41.0,False,False,True,True,False,21.0,87.0,0.0,0.0,10.0,4.0,6.0,9.0,0.0,1.0,0.0,"at least 8 dead, more inj Black church, white shooter\r\r\r\r\n12:50am Mayor says 9 killed, 1 injured Rev. Pickney was a SC senator, tried to kill self, out of ammo","Shot - Dead (murder, accidental, suicide)",Suicide - Attempt,119392.0,127815.0,93.410007,4625364,0.306302,32.7876,-79.9332,2.298900e+06,3.793572e+06,40,1.0,66.0,52.0,0.0,0.0,0.4,10.0,17,-0.299363,-0.954139,2,0.974928,-0.222521,6,1.224647e-16,-1.0,2015.0,897,14.6,0.0,13.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [9]:
mass_shooting_index = mass_shooting.index[0]
mass_shooting_pos = incidents_test_df.index.get_loc(mass_shooting_index)

selected_records_to_explain['indexes'].append(mass_shooting_index)
selected_records_to_explain['positions'].append(mass_shooting_pos)
selected_records_to_explain['instance names'].append('Mass shooting')
selected_records_to_explain['true labels'].append(true_labels_test[mass_shooting_pos])

## Incidents predicted as Fatal with highest probability

In [10]:
indeces_max_prob_death = []
for clf_name in clf_names:
    if clf_name != Classifiers.NC.value and clf_name != Classifiers.KNN.value:
        pos = preds[clf_name]['probs'].idxmax()
        indeces_max_prob_death.append(pos)

        selected_records_to_explain['indexes'].append(-1)
        selected_records_to_explain['positions'].append(pos)
        selected_records_to_explain['instance names'].append(f'Fatal with highest confidence by {clf_name}')
        selected_records_to_explain['true labels'].append(true_labels_test[pos])

max_prob_death_table = {}
for index in indeces_max_prob_death:
    max_prob_death_table[index] = {}
    max_prob_death_table[index]['True_label'] = true_labels_test[index]
    for clf_name in clf_names:
        if clf_name != Classifiers.NC.value and clf_name != Classifiers.KNN.value:
            max_prob_death_table[index][clf_name+'_pos_prob'] = preds[clf_name]['probs'][index]
max_prob_death_table = pd.DataFrame(max_prob_death_table).T
max_prob_death_table.style.background_gradient(cmap='Blues', axis=1)

,True_label,DecisionTreeClassifier_pos_prob,NeuralNetworkClassifier_pos_prob,RandomForestClassifier_pos_prob,NaiveBayesMixedClassifier_pos_prob,SupportVectorMachineClassifier_pos_prob,TabNetClassifier_pos_prob,ExtremeGradientBoostingClassifier_pos_prob
24457,0.000000,1.000000,0.275494,0.334883,0.506774,0.479334,0.145073,0.308241
22475,1.000000,0.983533,0.998547,0.964696,0.971509,0.525933,0.976280,0.962470
19289,1.000000,0.997163,0.978537,0.971552,0.975710,0.750369,0.971553,0.997538
102,1.000000,0.970151,0.993152,0.883908,0.996964,0.585770,0.912399,0.449456
12148,1.000000,0.919655,0.941554,0.804903,0.421525,0.992869,0.760588,0.950161
90,1.000000,0.980424,0.975513,0.945289,0.971846,0.792893,0.976280,0.988940
39930,1.000000,0.997163,0.985710,0.954709,0.978184,0.856635,0.975947,0.999680


In [11]:
pd.concat([
    max_prob_death_table.reset_index(),
    incidents_test_df.iloc[indeces_max_prob_death].reset_index()],
    axis=1
)

,index,True_label,DecisionTreeClassifier_pos_prob,NeuralNetworkClassifier_pos_prob,RandomForestClassifier_pos_prob,NaiveBayesMixedClassifier_pos_prob,SupportVectorMachineClassifier_pos_prob,TabNetClassifier_pos_prob,ExtremeGradientBoostingClassifier_pos_prob,index,date_original,state,address,county,city,address_type,state_house_district,state_senate_district,px_code,participant_age1,participant1_child,participant1_teen,participant1_adult,participant1_male,participant1_female,min_age,max_age,n_child,n_teen,n_adult,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,notes,incident_characteristics1,incident_characteristics2,candidate_votes,total_votes,candidate_perc,population_state_2010,location_imp,latitude,longitude,x,y,state_code,congressional_district,age_range,avg_age,n_child_prop,n_teen_prop,n_males_prop,n_participants,day,day_x,day_y,day_of_week,day_of_week_x,day_of_week_y,month,month_x,month_y,year,days_from_first_incident,poverty_perc,democrat,gun_law_rank,aggression,accidental,defensive,suicide,road,house,school,business,illegal_holding,drug_alcohol,officers,organized,social_reasons,abduction
0,24457,0.0,1.000000,0.275494,0.334883,0.506774,0.479334,0.145073,0.308241,90302,2017-08-06,IDAHO,3300 block of Lancer Ave,Ada County,Boise,building,15.0,15.0,ID,19.0,False,False,True,True,False,19.0,19.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,"shots fired following fight;\r\r\r\r\n43.634828, -116.341213 ;",Shots Fired - No Injuries,NaN,242252.0,355357.0,68.171444,1567582,1.000000e-05,43.6348,-116.3410,-8.995619e+05,4.979798e+06,12,1.0,0.0,19.0,0.0,0.0,1.000000,1.0,6,9.377521e-01,0.347305,6,-0.781831,0.623490,8,-8.660254e-01,-5.000000e-01,2017.0,1678,10.0,0.0,13.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,22475,1.0,0.983533,0.998547,0.964696,0.971509,0.525933,0.976280,0.962470,16951,2015-06-17,SOUTH CAROLINA,110 Calhoun Street,Charleston County,Charleston,amenity,111.0,42.0,SC,41.0,False,False,True,True,False,21.0,87.0,0.0,0.0,10.0,4.0,6.0,9.0,0.0,1.0,0.0,"at least 8 dead, more inj Black church, white shooter\r\r\r\r\n12:50am Mayor says 9 killed, 1 injured Rev. Pickney was a SC senator, tried to kill self, out of ammo","Shot - Dead (murder, accidental, suicide)",Suicide - Attempt,119392.0,127815.0,93.410007,4625364,3.063015e-01,32.7876,-79.9332,2.298900e+06,3.793572e+06,40,1.0,66.0,52.0,0.0,0.0,0.400000,10.0,17,-2.993631e-01,-0.954139,2,0.974928,-0.222521,6,1.224647e-16,-1.000000e+00,2015.0,897,14.6,0.0,13.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,19289,1.0,0.997163,0.978537,0.971552,0.975710,0.750369,0.971553,0.997538,128474,2014-10-07,ARKANSAS,4510 Thompson Road,Jefferson County,Pine Bluff,place,16.0,27.0,AR,68.0,False,False,True,True,False,64.0,68.0,0.0,0.0,2.0,1.0,1.0,2.0,0.0,0.0,0.0,husband and wife shot and killed; possible murder/suicide,"Shot - Dead (murder, accidental, suicide)",Suicide^,110789.0,206131.0,53.746889,2915918,1.000000e-07,34.1621,-92.0506,1.141124e+06,3.802027e+06,3,4.0,4.0,66.0,0.0,0.0,0.500000,2.0,7,9.884683e-01,0.151428,1,0.781831,0.623490,10,-8.660254e-01,5.000000e-01,2014.0,644,16.8,0.0,13.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,102,1.0,0.970151,0.993152,0.883908,0.996964,0.585770,0.912399,0.449456,220730,2016-03-31,HAWAII,73-1200 Mahilani Drive,Hawaiʻi County,NaN,road,6.0,4.0,HI,62.0,False,False,True,False,True,62.0,62.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,"Kona Highlands, self inflicted gunshot, init husband arrest for hom, ruled sui 5/2016\r\r\r\r\n19.727954, -155.980485","Shot - Dead (murder, accidental, suicide)",Suicide^,170848.0,224133.0,76.226169,1360301,1.000100e-01,19.7281,-155.9810,-6.326315e+06,3.706255e+06,11,2.0,0.0,62.0,0.0,0.0,0.000000,1.0,31,-2.449294e-16,1.000000,3,0.433884,-0.900969,3,1.000000e+00,6.123234e-17,2016.0,1185,9.5,1.0,3.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,12148,1.0,0.919655,0.941554,0.804903,0.421525,0.992869,0.760588,0.950161,62442,2017-03-26,OHIO,4601 Kellogg Ave,Hamilton County,Cincinnati,place,27.0,7.0,OH,27.0,False,False,True,True,False,24.0,34.0,0.0,0.0,18.0,13.0,4.0,2.0,15.0,1.0,0.0,"2 killed, 1

## Incidents predict as Non-Fatal with highest probability

In [12]:
indeces_min_prob_death = []
for clf_name in clf_names:
    if clf_name != Classifiers.NC.value and clf_name != Classifiers.KNN.value:
        pos = preds[clf_name]['probs'].idxmin()
        indeces_min_prob_death.append(pos)

        selected_records_to_explain['indexes'].append(-1)
        selected_records_to_explain['positions'].append(pos)
        selected_records_to_explain['instance names'].append(f'Non-Fatal with highest confidence by {clf_name}')
        selected_records_to_explain['true labels'].append(true_labels_test[pos])

min_prob_death_table = {}
for index in indeces_min_prob_death:
    min_prob_death_table[index] = {}
    min_prob_death_table[index]['True_label'] = true_labels_test[index]
    for clf_name in clf_names:
        if clf_name != Classifiers.NC.value and clf_name != Classifiers.KNN.value:
            min_prob_death_table[index][clf_name+'_pos_prob'] = preds[clf_name]['probs'][index]
min_prob_death_table = pd.DataFrame(min_prob_death_table).T
min_prob_death_table.style.background_gradient(cmap='Blues', axis=1)

,True_label,DecisionTreeClassifier_pos_prob,NeuralNetworkClassifier_pos_prob,RandomForestClassifier_pos_prob,NaiveBayesMixedClassifier_pos_prob,SupportVectorMachineClassifier_pos_prob,TabNetClassifier_pos_prob,ExtremeGradientBoostingClassifier_pos_prob
45,0.000000,0.000000,0.005506,0.068733,0.026357,0.055459,0.015196,0.002154
11801,0.000000,0.010353,0.000001,0.088031,0.016595,0.026304,0.005261,0.002582
38574,0.000000,0.000000,0.011187,0.039111,0.017209,0.051772,0.013530,0.001539
35589,0.000000,0.401427,0.006468,0.292325,0.002395,0.013763,0.289235,0.006366
4064,0.000000,0.202707,0.270518,0.422422,0.784434,0.000395,0.197593,0.011804
33985,0.000000,0.036537,0.002120,0.092084,0.004519,0.009118,0.000120,0.009118
20827,0.000000,0.104770,0.000267,0.129066,0.010176,0.002902,0.001796,0.000237


In [13]:
pd.concat([
    min_prob_death_table.reset_index(),
    incidents_test_df.iloc[indeces_min_prob_death].reset_index()],
    axis=1
)

,index,True_label,DecisionTreeClassifier_pos_prob,NeuralNetworkClassifier_pos_prob,RandomForestClassifier_pos_prob,NaiveBayesMixedClassifier_pos_prob,SupportVectorMachineClassifier_pos_prob,TabNetClassifier_pos_prob,ExtremeGradientBoostingClassifier_pos_prob,index,date_original,state,address,county,city,address_type,state_house_district,state_senate_district,px_code,participant_age1,participant1_child,participant1_teen,participant1_adult,participant1_male,participant1_female,min_age,max_age,n_child,n_teen,n_adult,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,notes,incident_characteristics1,incident_characteristics2,candidate_votes,total_votes,candidate_perc,population_state_2010,location_imp,latitude,longitude,x,y,state_code,congressional_district,age_range,avg_age,n_child_prop,n_teen_prop,n_males_prop,n_participants,day,day_x,day_y,day_of_week,day_of_week_x,day_of_week_y,month,month_x,month_y,year,days_from_first_incident,poverty_perc,democrat,gun_law_rank,aggression,accidental,defensive,suicide,road,house,school,business,illegal_holding,drug_alcohol,officers,organized,social_reasons,abduction
0,45,0.0,0.000000,0.005506,0.068733,0.026357,0.055459,0.015196,0.002154,166734,2018-03-07,MARYLAND,Hanover St and McComas St,NaN,Baltimore,place,46.0,46.0,MD,18.0,False,False,True,True,False,18.0,18.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,"pulled over for not using seat belt, stolen handgun recovered",Non-Shooting Incident,ATF/LE Confiscation/Raid/Arrest,202407.0,292880.0,69.109191,5773552,1.000000e-07,39.2662,-76.6148,2.440020e+06,4.593152e+06,20,3.0,0.0,18.0,0.0,0.0,1.000000,1.0,7,0.988468,0.151428,2,0.974928,-0.222521,3,1.000000,6.123234e-17,2018.0,1891,8.1,1.0,3.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
1,11801,0.0,0.010353,0.000001,0.088031,0.016595,0.026304,0.005261,0.002582,134816,2014-07-15,LOUISIANA,3100 block of Looney St.,Caddo Parish,Shreveport,place,2.0,39.0,LA,60.0,False,False,True,True,False,60.0,60.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,Kidnapping/abductions/hostage,Brandishing/flourishing/open carry/lost/found,152683.0,207919.0,73.433885,4533372,1.000000e-07,32.5027,-93.7866,9.900154e+05,3.608158e+06,18,4.0,0.0,60.0,0.0,0.0,1.000000,1.0,15,0.101168,-0.994869,1,0.781831,0.623490,7,-0.500000,-8.660254e-01,2014.0,560,20.6,0.0,13.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
2,38574,0.0,0.000000,0.011187,0.039111,0.017209,0.051772,0.013530,0.001539,113626,2016-04-05,MASSACHUSETTS,East Ashland and North Carey,Plymouth County,Brockton,building,NaN,NaN,MA,24.0,False,False,True,True,False,24.0,24.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,"traffic stop, Glock 9mm",Non-Shooting Incident,ATF/LE Confiscation/Raid/Arrest,271019.0,396009.0,68.437586,6547629,1.000000e-05,42.0961,-71.0092,2.820868e+06,5.057194e+06,21,8.0,0.0,24.0,0.0,0.0,1.000000,1.0,5,0.848644,0.528964,1,0.781831,0.623490,4,0.866025,-5.000000e-01,2016.0,1190,9.8,1.0,3.0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
3,35589,0.0,0.401427,0.006468,0.292325,0.002395,0.013763,0.289235,0.006366,115738,2014-03-02,ALASKA,Farmers Loop Road,Fairbanks North Star,NaN,place,2.0,NaN,AK,19.0,False,False,True,True,False,18.0,19.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0,at Sourdough Fuel store,Armed robbery with injury/death and/or evidence of DGU found,Brandishing/flourishing/open carry/lost/found,142572.0,279741.0,50.965715,710231,1.000000e-07,64.8893,-147.7590,-1.613860e+06,8.082339e+06,1,0.0,1.0,18.0,0.0,0.0,1.000000,5.0,2,0.394356,0.918958,6,-0.781831,0.623490,3,1.000000,6.123234e-17,2014.0,425,11.2,0.0,13.0,1,0,0,0,0,0,0,1,1,0,0,0,0,0
4,4064,0.0,0.202707,0.270518,0.422422,0.784434,0.000395,0.197593,0.011804,210767,2013-05-12,LOUISIANA,Frenchmen Street,Orleans Parish,New Orleans,place,97.0,4.0,LA,NaN,NaN,NaN,NaN,NaN,NaN,10.0,23.0,0.0,0.0,22.0,14.0,8.0,0.0,19.0,3.0,0.0,ms; 19 inj. Rival gang members retaliation effort. Akein and Shawn Scott were shooters. Stanley Scott ordered hit. 3 brothers sentenced.,Shot - Wounded/Injured,"Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one loc

In [14]:
selected_records_df = pd.DataFrame(selected_records_to_explain)
selected_records_df.to_csv('../data/explanation_results/selected_records_to_explain.csv')
selected_records_df

,indexes,positions,instance names,true labels
0,178231,243,Attempted Suicide,0
1,16951,22475,Mass shooting,1
2,-1,24457,Fatal with highest confidence by DecisionTreeClassifier,0
3,-1,22475,Fatal with highest confidence by NeuralNetworkClassifier,1
4,-1,19289,Fatal with highest confidence by RandomForestClassifier,1
5,-1,102,Fatal with highest confidence by NaiveBayesMixedClassifier,1
6,-1,12148,Fatal with highest confidence by SupportVectorMachineClassifier,1
7,-1,90,Fatal with highest confidence by TabNetClassifier,1
8,-1,39930,Fatal with highest confidence by ExtremeGradientBoostingClassifier,1
9,-1,45,Non-Fatal with highest confidence by DecisionTreeClassifier,0


In [15]:
random_records_to_explain = {}
random_records_to_explain['positions'] = np.arange(0, 51)
random_records_to_explain['true labels'] = true_labels_test[0: 51]
random_records_df = pd.DataFrame(random_records_to_explain)
random_records_df.to_csv('../data/explanation_results/random_records_to_explain.csv')